In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import StandardScaler

In [7]:
# Lendo o arquivo com o delimitador correto e codificação
df = pd.read_csv("https://datalaketech4.blob.core.windows.net/dados-ipea/cotacao_petroleo_ipea.csv", encoding="ISO-8859-1", delimiter=";", decimal=",")

In [8]:
df.head()

,data,valor_us$
0,01/01/2015,55.27
1,02/01/2015,57.86
2,03/01/2015,55.60
3,04/01/2015,55.27
4,05/01/2015,55.27


In [ ]:
#Ajuste na data 
df.columns = df.columns.str.strip() 
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y') 

In [10]:
#Exibindo primeiras linhas 
print(df.head())

        data  valor_us$
0 2015-01-01      55.27
1 2015-01-02      57.86
2 2015-01-03      55.60
3 2015-01-04      55.27
4 2015-01-05      55.27


In [ ]:
#Tratando dados 
print(df.isnull().sum())  # Contar valores nulos
df.dropna(inplace=True)  # Remover se necessário

In [ ]:
#Verificar e remover outliers
plt.boxplot(df['preco'])
plt.title("Detecção de Outliers no Preço do Petróleo")
plt.show()

In [ ]:
# Calcular o IQR para detectar outliers
Q1 = df['preco'].quantile(0.25)
Q3 = df['preco'].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
# Definir limites para valores aceitáveis
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

In [ ]:
# Filtrar outliers
df_sem_outliers = df[(df['preco'] >= limite_inferior) & (df['preco'] <= limite_superior)]

In [ ]:
# Exibir gráfico novamente após a remoção dos outliers
plt.boxplot(df_sem_outliers['preco'])
plt.title("Detecção de Outliers no Preço do Petróleo (sem outliers)")
plt.show()


In [ ]:
#Removendo valores 
df = df[df['preco'] < df['preco'].quantile(0.99)]  # Exemplo de remoção de outliers acima do percentil 99

In [ ]:
#Evolução do preço ao longo do tempo
plt.figure(figsize=(12,6))
sns.lineplot(x=df['data'], y=df['preco'])
plt.title("Variação do Preço do Petróleo Brent ao Longo do Tempo")
plt.xlabel("Data")
plt.ylabel("Preço (USD)")
plt.grid()
plt.show()

In [ ]:
#Estatísticas descritivas para entender a distribuição dos preços
print(df['preco'].describe())

In [ ]:
#Componentes sazonais
df['ano'] = df['data'].dt.year
df['mes'] = df['data'].dt.month
df['dia_da_semana'] = df['data'].dt.dayofweek  # Segunda = 0, Domingo = 6

In [ ]:
#Médias móveis para suavizar oscilações
df['media_movel_30'] = df['preco'].rolling(window=30).mean()

In [ ]:
# Criando uma variável de lag de 1 dia
df['preco_lag_1'] = df['preco'].shift(1)

In [ ]:
# Calculando a variação percentual
df['variacao_percentual'] = df['preco'].pct_change() * 100

In [ ]:
# Normalizando os preços
scaler = StandardScaler()
df['preco_normalizado'] = scaler.fit_transform(df[['preco']])

In [ ]:
# Calculando o logaritmo do preço
df['preco_log'] = np.log(df['preco'])

In [ ]:
# Preenchendo os valores NaN
df['media_movel_30'] = df['media_movel_30'].fillna(df['media_movel_30'].mean())
df['variacao_percentual'] = df['variacao_percentual'].fillna(df['variacao_percentual'].mean())


In [ ]:
# Verificando ajuste
print(df.head())

In [ ]:
# Verificando a correlação entre as variáveis
print(df.corr())

In [ ]:
# Gráfico de dispersão entre preço no dia anterior e preço logarítmico
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df['preco_lag_1'], y=df['preco_log'])
plt.title('Correlação entre Preço Logarítmico e Preço no Dia Anterior')
plt.xlabel('Preço no Dia Anterior')
plt.ylabel('Preço Logarítmico')
plt.show()

In [ ]:
# Decomposição da série temporal
decomposicao = seasonal_decompose(df['preco'], model='multiplicative', period=365)
decomposicao.plot()
plt.show()

In [ ]:
# Teste de Dickey-Fuller para verificar estacionariedade
result = adfuller(df['preco'])
print(f'Estatística ADF: {result[0]}')
print(f'p-valor: {result[1]}')